<a href="https://colab.research.google.com/github/Uriel1201/HelloSQL/blob/main/08_upgrade_rate/08_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 8. Upgrade rate by product action (hard)
Given the following two tables, return the fraction of users, rounded to two decimal places, who accessed feature two (type: F2 in events table) and upgraded to premium within the first 30 days of signing up.

In [65]:
import pandas as pd
import numpy  as np

data1 = {'user_id'   : [1,2,3,4,5,6,7],
         'name'      : ['John','Jane','Jill','Josh','Jean','Justin','Jeremy'],
         'join_date' : ['14-feb-20',
                        '14-feb-20',
                        '15-feb-20',
                        '15-feb-20',
                        '16-feb-20',
                        '17-feb-20',
                        '18-feb-20']
        }

data2 = {'user_id'     : [1,2,2,3,4,1,3],
         'Type'        : ['F1','F2','P','F2','F2','P','P'],
         'access_date' : ['1-mar-20',
                          '2-mar-20',
                          '12-mar-20',
                          '15-mar-20',
                          '15-mar-20',
                          '16-mar-20',
                          '22-mar-20']
        }

users  = pd.DataFrame(data1)
events = pd.DataFrame(data2)

In [66]:
users['join_date'] = pd.to_datetime(users['join_date'])
events['access_date'] = pd.to_datetime(events['access_date'])

<ipython-input-66-014f4516001d>:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  users['join_date'] = pd.to_datetime(users['join_date'])
<ipython-input-66-014f4516001d>:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  events['access_date'] = pd.to_datetime(events['access_date'])


In [67]:
users

,user_id,name,join_date
0,1,John,2020-02-14
1,2,Jane,2020-02-14
2,3,Jill,2020-02-15
3,4,Josh,2020-02-15
4,5,Jean,2020-02-16
5,6,Justin,2020-02-17
6,7,Jeremy,2020-02-18


In [68]:
events

,user_id,Type,access_date
0,1,F1,2020-03-01
1,2,F2,2020-03-02
2,2,P,2020-03-12
3,3,F2,2020-03-15
4,4,F2,2020-03-15
5,1,P,2020-03-16
6,3,P,2020-03-22


In [69]:
def row_num(x):
      x_s = pd.Series(x)
      row_n = range(1, len(x_s) + 1)
      return row_n

In [70]:
events['access_rank'] = events.groupby('user_id')['access_date'].transform(row_num)

In [71]:
events

,user_id,Type,access_date,access_rank
0,1,F1,2020-03-01,1
1,2,F2,2020-03-02,1
2,2,P,2020-03-12,2
3,3,F2,2020-03-15,1
4,4,F2,2020-03-15,1
5,1,P,2020-03-16,2
6,3,P,2020-03-22,2


In [72]:
c1 = events['Type'] == 'F2'
c2 = events['access_rank'] == 1
first_F2 = events.loc[c1 & c2, ['user_id', 'access_date']]

In [73]:
first_F2

,user_id,access_date
1,2,2020-03-02
3,3,2020-03-15
4,4,2020-03-15


In [74]:
first_F2.set_index('user_id', inplace = True)
users.set_index('user_id', inplace = True)

In [75]:
first_F2

,access_date
user_id,
2,2020-03-02
3,2020-03-15
4,2020-03-15


In [76]:
users

,name,join_date
user_id,,
1,John,2020-02-14
2,Jane,2020-02-14
3,Jill,2020-02-15
4,Josh,2020-02-15
5,Jean,2020-02-16
6,Justin,2020-02-17
7,Jeremy,2020-02-18


In [77]:
c = events['Type'] == 'P'
premium_users = events.loc[c, ['user_id', 'access_date']]

In [78]:
premium_users.set_index('user_id', inplace = True)
premium_users.rename(columns = {'access_date' : 'access_premium_date'}, inplace = True)

In [80]:
premium_users

,access_premium_date
user_id,
2,2020-03-12
1,2020-03-16
3,2020-03-22


In [81]:
F2_users = pd.concat([users, first_F2], axis = 1, join = 'inner')

In [82]:
F2_users

,name,join_date,access_date
user_id,,,
2,Jane,2020-02-14,2020-03-02
3,Jill,2020-02-15,2020-03-15
4,Josh,2020-02-15,2020-03-15


In [83]:
F2_toP = pd.merge(F2_users, premium_users, left_index = True, right_index = True, how = 'left')

In [84]:
F2_toP

,name,join_date,access_date,access_premium_date
user_id,,,,
2,Jane,2020-02-14,2020-03-02,2020-03-12
3,Jill,2020-02-15,2020-03-15,2020-03-22
4,Josh,2020-02-15,2020-03-15,NaT


In [85]:
i = (np.where(F2_toP['access_premium_date'] - F2_toP['join_date'] <= pd.to_timedelta('30 days'), 1, 0).sum() / len(F2_toP.index)).round(2)
d = {'upgrade_rate' : [i]
    }
upgrade = pd.DataFrame(data = d, index = [0])
upgrade

,upgrade_rate
0,0.33
